In [82]:
import pandas as pd
#import essay
import numpy as np
import pickle
import re

In [83]:
data = pd.read_csv("mypersonality_final.csv",encoding='cp1252', delimiter=',', quotechar='"')

In [84]:
data = data[['STATUS','cEXT','cNEU','cAGR','cCON','cOPN']]

In [85]:
data.head()

,STATUS,cEXT,cNEU,cAGR,cCON,cOPN
0,likes the sound of thunder.,n,y,n,n,y
1,is so sleepy it's not even funny that's she ca...,n,y,n,n,y
2,is sore and wants the knot of muscles at the b...,n,y,n,n,y
3,likes how the day sounds in this new song.,n,y,n,n,y
4,is home. <3,n,y,n,n,y


In [86]:
for e in data.columns[1:6]:
    data[e] = data[e].replace('n', '0')
    data[e] = data[e].replace('y', '1')
    data[e] = pd.to_numeric(data[e])

data.head()

,STATUS,cEXT,cNEU,cAGR,cCON,cOPN
0,likes the sound of thunder.,0,1,0,0,1
1,is so sleepy it's not even funny that's she ca...,0,1,0,0,1
2,is sore and wants the knot of muscles at the b...,0,1,0,0,1
3,likes how the day sounds in this new song.,0,1,0,0,1
4,is home. <3,0,1,0,0,1


In [87]:
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

stop_words = stopwords.words("english")

def preprocess(text):
    corpus = []
    
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token)>2 and token not in stop_words:
            corpus.append(token)
    return corpus

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Piyush\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Piyush\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [88]:
data['clean'] = data['STATUS'].apply(preprocess)
data['clean_text'] = data['clean'].apply(lambda x:" ".join(x))
data.head()

,STATUS,cEXT,cNEU,cAGR,cCON,cOPN,clean,clean_text
0,likes the sound of thunder.,0,1,0,0,1,"[likes, sound, thunder]",likes sound thunder
1,is so sleepy it's not even funny that's she ca...,0,1,0,0,1,"[sleepy, funny, sleep]",sleepy funny sleep
2,is sore and wants the knot of muscles at the b...,0,1,0,0,1,"[sore, wants, knot, muscles, base, neck, stop,...",sore wants knot muscles base neck stop hurting...
3,likes how the day sounds in this new song.,0,1,0,0,1,"[likes, day, sounds, new, song]",likes day sounds new song
4,is home. <3,0,1,0,0,1,[home],home


In [89]:
cNEU = pickle.load( open("pickles/cNEU_project.p", "rb"))
cAGR = pickle.load( open("pickles/cAGR_project (1).p", "rb"))
cCON = pickle.load( open("pickles/cCON_project (1).p", "rb"))
cOPN = pickle.load( open("pickles/cOPN_project (1).p", "rb"))
cEXT = pickle.load( open("pickles/cEXT_project (1).p", "rb"))

c:\users\piyush\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator SVC from version 0.22.2.post1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
c:\users\piyush\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.2.post1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [90]:
with open('pickles/bow_vectorizer_project.p', 'rb') as f1:
    bow_transformer = pickle.load(f1)

c:\users\piyush\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.2.post1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [91]:
with open('pickles/tfidf_vectorizer_project (1).p', 'rb') as f:
    tfidf_transformer = pickle.load(f)

c:\users\piyush\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.22.2.post1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
c:\users\piyush\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.22.2.post1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [92]:
def predict_personality(text):
    sentences = re.split("(?<=[.!?]) +", text)
    text_vector_31 = tfidf_transformer.transform(sentences)
    text_vector_32 = bow_transformer.transform(sentences)
    EXT = cEXT.predict(text_vector_31)
    AGR = cAGR.predict(text_vector_31)
    CON = cCON.predict(text_vector_31)
    OPN = cOPN.predict(text_vector_31)
    return [EXT[0]]


In [93]:
def predict_final(text):
  predictions = predict_personality(text)
  ll = []
  if predictions[0] == 0:
    ll.append('Not Extraversion')
  elif predictions[0] == 1:
    ll.append('Extraversion')

  if predictions[1] == 0:
    ll.append('Not Neuroticism')
  elif predictions[1] == 1:
    ll.append('Neuroticism')

  if predictions[2] == 0:
    ll.append('Not Agreeableness')
  elif predictions[2] == 1:
    ll.append('Agreeableness')

  if predictions[3] == 0:
    ll.append('Not Conscientiousness')
  elif predictions[3] == 1:
    ll.append('Conscientiousness')
  
  if predictions[4] == 0:
    ll.append('Not Openness')
  elif predictions[4] == 1:
    ll.append('Openness')
  
  return ll

In [94]:
data['clean_text']

0                                     likes sound thunder
1                                      sleepy funny sleep
2       sore wants knot muscles base neck stop hurting...
3                               likes day sounds new song
4                                                    home
                              ...                        
9912                                   little things away
9913                                     wishing saturday
9914                                        studying hard
9915                                         snipers head
9916    night amazing propname propname performance re...
Name: clean_text, Length: 9917, dtype: object

In [95]:
from sklearn.model_selection import train_test_split
training, test = train_test_split(data, test_size=0.20, random_state=42)

In [96]:
train_x = training.clean_text

train_y_cEXT = training['cEXT']
train_y_cNEU = training['cNEU']
train_y_cAGR = training['cAGR']
train_y_cCON = training['cCON']
train_y_cOPN = training['cOPN']


test_x = test.clean_text

test_y_cEXT = test['cEXT']
test_y_cNEU = test['cNEU']
test_y_cAGR = test['cAGR']
test_y_cCON = test['cCON']
test_y_cOPN = test['cOPN']

In [97]:
# for evaluation save some data for later:
evaluation = []
evaluation_tf = []
data_length = len(data)
vec_name = "MBTI"

In [98]:
train_x_vectors = bow_transformer.transform(train_x)
test_x_vectors = bow_transformer.transform(test_x)

In [99]:
train_x_vectors_tf = tfidf_transformer.transform(train_x)
test_x_vectors_tf = tfidf_transformer.transform(test_x)

In [101]:
EXT = cEXT.predict(train_x_vectors_tf)

In [105]:
EXT

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [106]:
from sklearn.metrics import accuracy_score
ac = accuracy_score(train_y_cEXT, EXT)

In [107]:
ac

0.554519097441069

In [108]:
OPN = cOPN.predict(train_x_vectors_tf)

In [110]:
from sklearn.metrics import accuracy_score
ac = accuracy_score(train_y_cOPN, OPN)
ac

0.7224253119878986

In [115]:
AGR = cAGR.predict(train_x_vectors_tf)
from sklearn.metrics import accuracy_score
ac = accuracy_score(train_y_cAGR, AGR)
ac

0.48329761754695577

In [116]:
CON = cCON.predict(train_x_vectors_tf)
from sklearn.metrics import accuracy_score
ac = accuracy_score(train_y_cCON, CON)
ac

0.5406529686121265